## Import Statements

In [3]:
import gc
import os
import sys
import random
import pickle
import bisect
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

os.environ["CDF_LIB"] = "/usr/local/lib/cdf/lib/"
import netCDF4 as nc
import pyspedas
from spacepy import pycdf
from pyspedas import wind
from pytplot import tplot, get_data

## Various functions

In [4]:
# Constants
e = 1.602176634e-19
m = 9.109383701528e-31
e_0 = 8.854187812813e-12
cons = e/((m*e_0)**.5)/(2*np.pi)

In [5]:
def find_closest_time(time, lst):
    return min(range(len(lst)), 
               key=lambda i: abs(time-lst[i]))

def reduced_time_closest_time(goal_time, time_lst):
    """
    Find the index in the availible times which is closest to the goal time
    """
    # Bisect gives the index of the smallest value which is greater than goal time
    upper_index = min(bisect.bisect_left(time_lst, goal_time), len(time_lst)-1)
    lower_index = max(upper_index-1, 0)
    if abs(time_lst[upper_index] - goal_time) > abs(time_lst[lower_index] - goal_time):
        return lower_index
    else:
        return upper_index

def conv_dens_to_freq(ne):
    assert ne>=0
    return ne**.5 * cons

def norm(lst, max_val):
    return np.array([val/max_val for val in lst])

def denorm(lst, max_val):
    return np.array([val*max_val for val in lst])

def print_lst(lst, sort=False):
    if sort:
        lst.sort()
    print("[")
    for n in lst:
        print(f"{repr(n)},")
    print("]")

def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)


## Importing Data

### pySPEDAS

In [9]:
pyspedas_data = wind.swe(trange=['1995-06-01', '1995-06-27'], datatype='k0')

01-Jul-22 14:23:32: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/wind/swe/swe_k0/1995/
01-Jul-22 14:23:33: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950601_v02.cdf
01-Jul-22 14:23:33: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950602_v02.cdf
01-Jul-22 14:23:33: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950603_v02.cdf
01-Jul-22 14:23:34: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950604_v02.cdf
01-Jul-22 14:23:34: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950605_v02.cdf
01-Jul-22 14:23:34: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950606_v02.cdf
01-Jul-22 14:23:34: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950607_v02.cdf
01-Jul-22 14:23:35: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950608_v02.cdf
01-Jul-22 14:23:35: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950609_v02.cdf
01-Jul-22 14:23:35: File is current: wind_data/swe/swe_k0/1995/wi_k0_swe_19950610_v02.

In [10]:
plt.plot(data.times,data.y)
plt.xlabel("km/s")
plt.ylabel("time in epoch")
plt.savefig("plots/figure.png")

AttributeError: 'NoneType' object has no attribute 'times'

In [11]:
data = get_data("V_GSM")
raw_vsw = abs(data.y[:,0])
time_vsw = data.times[:]

In [87]:
# plt.plot(data.times, (data.y[:,0]**2 + data.y[:,0]**2 +data.y[:,0]**2)**.5, label="V_GSM - sqrt(x^2 + y^2 + z^2)")
# plt.plot(data.times, abs(data.y[:,0]), label="V_GSM - abs(x)")

# plt.axhline(y=750, label="Fast Wind", color='g')
# plt.axhline(y=500, label="Slow Wind Maximum", color='r')
# plt.axhline(y=300, label="Slow Wind Minimum", color='b')
# plt.legend()
# plt.title("Figuring out what to use for solar wind speed")
# plt.xlabel("Epoch Time (s)")
# plt.ylabel("Solar Wind Speed (km/s)")
# plt.savefig("plots/vsw_k0.png")
# print(len(data.times), 26*24*3600/len(data.times))

### Solaris and CDPP data

In [12]:
spectra_folder = "data/solaris/1995-06/"
nn_folder = "data/cdpp/34069/DA_TC_WIND_WAVES_TNR_L3_NN/"

spectra_data = [ spectra_folder+file for file in os.listdir(spectra_folder) if file.endswith(".nc")]
nn_data = [ nn_folder+file for file in os.listdir(nn_folder) if file.endswith(".nc")]
spectra_data.sort()
nn_data.sort()
combined_data = zip(spectra_data, nn_data)
#print_lst(combined_data)

In [13]:
time = []
freq = []
spec = []
time2 = []
raw_fpe = []
raw_electron_density = []
for spectra_file_name, nn_file_name in combined_data:
    date = spectra_file_name[32:-3]
    date_epoch = int(datetime.datetime(int(date[0:4]), int(date[4:6]), int(date[6:8]), 0, 0, 0, 
                                       tzinfo=datetime.timezone.utc).timestamp())
    
    with nc.Dataset(spectra_file_name) as file:
        temp_time = file["tnr_ch1_time"][:]
        temp_spec = file["tnr_ch1_spec"][:]
        if len(freq) == 0:
            freq  = file["tnr_ch1_freq"][:]

    
    with nc.Dataset(nn_file_name) as file:
        temp_time2   = file["Time"][:]
        temp_raw_fpe = file["Plasma Frequency"][:]
        temp_dens    = file["Electron Density"][:]
        
        # Found slight bug where the times over some days are shifted by 86400 seconds up
        if temp_time2[10] >= 86400:
            print(f"Correced: {spectra_file_name}")
            temp_time2 = np.subtract(temp_time2, 86400)
        temp_time2 = np.add(temp_time2, date_epoch)

    if not temp_time[0] in time:
        time.extend(temp_time)
        spec.extend(np.transpose(temp_spec))
        time2.extend(temp_time2)
        raw_fpe.extend(temp_raw_fpe)
        raw_electron_density.extend(temp_dens)
        
    else:
        print(spectra_file_name)
print(len(time))

Correced: data/solaris/1995-06/wi_wav_tnr_19950603.nc
Correced: data/solaris/1995-06/wi_wav_tnr_19950604.nc
Correced: data/solaris/1995-06/wi_wav_tnr_19950611.nc
Correced: data/solaris/1995-06/wi_wav_tnr_19950618.nc
Correced: data/solaris/1995-06/wi_wav_tnr_19950619.nc
505726


In [6]:
pd.DataFrame([abs(time[i]-time[i-1]) for i in range(1, len(time))]).describe()

,0
count,505725.000000
mean,4.450767
std,3.454287
min,0.472000
25%,4.416000
50%,4.416000
75%,4.416000
max,1594.176000


In [14]:
# Increase memory efficiency
time2, raw_fpe, raw_electron_density = list(zip(*[d for d in zip(time2, raw_fpe, raw_electron_density) if d[1]>1]))

fpe = [raw_fpe[reduced_time_closest_time(t, time2)] for t in time]
electron_density = [raw_electron_density[reduced_time_closest_time(t, time2)] 
                    for t in time]

vsw = [raw_vsw[reduced_time_closest_time(t, time_vsw)] for t in time]
#plt.plot(raw_fpe)

In [15]:
print(np.shape(time), np.shape(spec), np.shape(fpe), np.shape(electron_density), np.shape(vsw))
data_v1 = list(zip(time, spec, fpe, electron_density, vsw))

(505726,) (505726, 96) (505726,) (505726,) (505726,)


## Randomizing and sorting data

In [16]:
random.seed(10)
random_index = list(range(len(data_v1)))
random.shuffle(random_index)
data_v3 = [data_v1[i] for i in random_index]    

In [17]:
max_y = 50 #max([d[2] for d in data_v3])
s = int(len(data_v3)*.9//1)
print(np.shape(data_v3[:s]))


# Takes the training section of the data and unzips the test and training data
train_x_data = np.array([d[1] for d in data_v3[:s]])
train_y_data = np.array([d[2]/max_y for d in data_v3[:s]])
train_y_data_ed = np.array([d[3] for d in data_v3[:s]])
train_y_data_vsw =  np.array([d[4] for d in data_v3[:s]])
print("Done with creating trainning data")

# Sorts test data so we could use that sorted data to make nice graphs at the end
test_sorted = data_v3[s:]
test_sorted.sort(key=lambda x: x[0])
print("Done sorting")

test_time_sorted = np.array([d[0] for d in test_sorted])
test_x_sorted = np.array([d[1] for d in test_sorted])
test_y_sorted = np.array([d[2]/max_y for d in test_sorted])
test_y_sorted_ed = np.array([d[3] for d in test_sorted])
test_y_sorted_vsw =  np.array([d[4] for d in test_sorted])

print("Done with creating test data")

01-Jul-22 14:24:23: /home/ayushmaan/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape



(455153, 5)
Done with creating trainning data
Done sorting
Done with creating test data


In [18]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


                  train_x_data: 333.4 MiB
                 test_x_sorted: 37.0 MiB
                          time:  4.3 MiB
                          spec:  4.3 MiB
                           fpe:  4.0 MiB
              electron_density:  4.0 MiB
                           vsw:  4.0 MiB
                       data_v1:  4.0 MiB
                       data_v3:  4.0 MiB
                  random_index:  3.9 MiB


In [19]:
data_dic = {}
data_dic["freq"]             = freq
data_dic["max_y"]            = max_y
data_dic["train_x_data"]     = train_x_data
data_dic["train_y_data"]     = train_y_data
data_dic["train_y_data_ed"]     = train_y_data_ed
data_dic["test_time_sorted"] = test_time_sorted
data_dic["test_x_sorted"]    = test_x_sorted
data_dic["test_y_sorted"]    = test_y_sorted
data_dic["test_y_sorted_ed"]    = test_y_sorted_ed
data_dic["test_y_sorted_vsw"] = test_y_sorted_vsw

In [13]:
455153/32*.9

12801.178125

In [21]:
with open("data/pickle/data", 'wb') as file:
    pickle.dump(data_dic, file=file)